In [2]:
import pickle
import pandas as pd
from datasets import Dataset, DatasetDict
import multiprocessing
import copy
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch,transformers
from reference.llm_loader import reload_model_and_tokenizer
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer
from reference.utils import get_tokenizer
from tqdm import tqdm
import logging
import os
import statistics
import random
random.seed(42)  # 设置随机种子以确保结果可复现

reload_path = "./models/second-stage-llama3.2-3b/checkpoints/long_texts_match/checkpoint-1011"


🌟 构造smiles_to_struct_code的映射

🌟 构造训练/验证/测试数据

In [2]:
map1 = {
    "p_np": (
        "Does this molecule have blood-brain barrier permeability (BBB penetration)? True for BBB permeable and False for not BBB permeable.",
        "True",
        "False"
    )
}
map2 = {
    "NR-AR": (
        "Does this molecule activate the androgen receptor (NR-AR)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-AR-LBD": (
        "Does this molecule activate the ligand-binding domain of the androgen receptor (NR-AR-LBD)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-AhR": (
        "Does this molecule activate the aryl hydrocarbon receptor (NR-AhR)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-Aromatase": (
        "Does this molecule inhibit the aromatase enzyme (NR-Aromatase)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-ER": (
        "Does this molecule activate the estrogen receptor (NR-ER)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-ER-LBD": (
        "Does this molecule activate the ligand-binding domain of the estrogen receptor (NR-ER-LBD)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-PPAR-gamma": (
        "Does this molecule activate the peroxisome proliferator-activated receptor gamma (NR-PPAR-gamma)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-ARE": (
        "Does this molecule activate the antioxidant response element (SR-ARE)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-ATAD5": (
        "Does this molecule activate ATAD5 signaling (SR-ATAD5)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-HSE": (
        "Does this molecule activate the heat shock element (SR-HSE)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-MMP": (
        "Does this molecule activate the mitochondrial membrane potential stress response (SR-MMP)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-p53": (
        "Does this molecule activate the p53 stress response pathway (SR-p53)? True for active and False for inactive.",
        "True",
        "False"
    )
}
map3 = {
    "FDA_APPROVED": (
        "Has this molecule been approved by the FDA? True for FDA approved and false for non-approved.",
        "True",
        "False"
    ),
    "CT_TOX": (
        "Is this molecule associated with clinical toxicity? True for clinically toxic and false for non-toxic.",
        "True",
        "False"
    )
}
map4 = {
    "HIV_active": (
        "Does this molecule inhibit HIV replication? True for active molecules which can inhibit HIV and false for inactive ones.",
        "True",
        "False"
    )
}
map5 = {
    "Class": (
        "Is the binding result of the molecular on beta-secretase 1 true or false?",
        "True",
        "False"
    )
}
# for dname,mapping in zip(['BBBP', 'Tox21', 'ClinTox', 'HIV', 'BACE'],[map1,map2,map3,map4,map5]):
#     print(dname,len(mapping))

# 将多个mapping合为一个
all_mapping = {**map1, **map2, **map3, **map4, **map5}
# print(len(all_mapping))

save_path = "./phase1/datasets/all/codes_map.pt"
codes_map = torch.load(save_path,weights_only=True)
label_desc_map = all_mapping

datasets_to_load = ['BBBP', 'Tox21', 'ClinTox', 'HIV', 'BACE']
prompt_template = """
You are a chemistry expert. Classify the given molecule into the correct category based on its molecular structure (token) and smiles expression. Each structure token represents a unique graph pattern (e.g., a kind of similar molecular graphs).
[Molecule] {text_attribute}
[Structure Token] <struct>{code}</struct>
[Task] {task_desc} Output the complete correct answer from the following two options:
1. {category_1}
2. {category_2}
[Answer]"""

tool,assistant_name = get_tokenizer(tokenizer_path=reload_path)
def process(example):
    # smiles example['text']
    text_attribute = example['smiles']
    task = example['task']
    code_number = example['code']
    code = f'<gstruct_{code_number}>'
    task_details = label_desc_map[task]
    example['instruction'] = prompt_template.format(
        text_attribute = text_attribute,
        code = code,
        task_desc = task_details[0],
        category_1 = task_details[1],        
        category_2 = task_details[2]
    )
    example['response'] = [task_details[1] if example['label'] == 1 else task_details[2]][0]
    messages = [
        {"role": "user", "content": example['instruction']},
        {"role": assistant_name, "content": example['response']},
    ]
    example["text"] = tool.apply_chat_template(messages, tokenize=False)+"<|end_of_text|>"
   
    return example

def seal_datasets(split='train'):
    datasets = {}
    start = 0
    for dataset_name in datasets_to_load:
        dataset_path = f"./phase1/datasets/{dataset_name}/graph_attributes.pkl"
        with open(dataset_path, "rb") as f:
            graphs = pickle.load(f)
        tar_graphs = graphs[split]

        tar_len = len(tar_graphs)
        tar_codes = list(codes_map[split].values())[start: start + tar_len]
        start += tar_len
        assert len(tar_codes) == tar_len,'Length mismatch between tar_graphs and tar_codes'
        for g,code in zip(tar_graphs,tar_codes):
            g['code'] = code
        
        tasks = g['label'].keys()
        for task in tasks:
            # print('Processing '+task+' for dataset: ',dataset_name)
            new_graphs = copy.deepcopy(tar_graphs)
            for g in new_graphs:
                g['label'] = g['label'][task]
                g['task'] = task

            if split == 'train' and dataset_name != 'BACE':
                print('Balancing training split for dataset: ',dataset_name, 'task', task)
                pos_samples = [g for g in new_graphs if g['label'] == 1]
                neg_samples = [g for g in new_graphs if g['label'] == 0]
                num_pos, num_neg = len(pos_samples), len(neg_samples)
                num_total = num_pos + num_neg
                print(f"原始: Total={num_total}, Pos={num_pos} , Neg={num_neg}")

                if num_pos < num_neg:
                    # 复制正样本
                    target_len = int(num_neg / 5)
                    repeat_factor = target_len // num_pos
                    remainder = target_len % num_pos
                    pos_samples_extended = pos_samples * repeat_factor + random.sample(pos_samples, remainder)
                    print(f"复制正样本: {num_pos} -> {len(pos_samples_extended)}")
                    balanced_graphs = pos_samples_extended + neg_samples
                    random.shuffle(balanced_graphs)
                elif num_neg < num_pos:
                    # 复制负样本
                    target_len = int(num_pos / 5)
                    repeat_factor = target_len // num_neg 
                    remainder = target_len % num_neg 
                    if dataset_name == 'BBBP':
                        repeat_factor = num_pos // num_neg
                        remainder = num_pos % num_neg  
                    neg_samples_extended = neg_samples * repeat_factor + random.sample(neg_samples, remainder)
                    print(f"复制负样本: {num_neg} -> {len(neg_samples_extended)}")
                    balanced_graphs = pos_samples + neg_samples_extended
                    random.shuffle(balanced_graphs)
                else:
                    balanced_graphs = new_graphs
            else:
                balanced_graphs = new_graphs

            raw_dataset = Dataset.from_list(balanced_graphs)
            raw_dataset = raw_dataset.add_column("smiles", raw_dataset["text"])
            raw_dataset = raw_dataset.remove_columns("text")

            print(f"Processing {dataset_name}_{task} split: {split}")

            datasets[dataset_name+'_'+task] = raw_dataset.map(
                process,
                num_proc= multiprocessing.cpu_count(),
                load_from_cache_file=False,
                remove_columns=raw_dataset.column_names,
            )
            # === 统计正负样本数量和比例 ===
            if split == 'train':
                labels = raw_dataset["label"]
                pos = sum(1 for l in labels if l == 1)
                neg = sum(1 for l in labels if l == 0)
                total = len(labels)
                pos_ratio = pos / total if total > 0 else 0
                neg_ratio = neg / total if total > 0 else 0
                print(f"{dataset_name}_{task}(TRAIN): Total={total}, Pos={pos} ({pos_ratio:.2%}), Neg={neg} ({neg_ratio:.2%})")

    return datasets

splits = ['train','valid','test']


ds_dict = {}
for split in splits:
    ds_dict[split] = seal_datasets(split)

# 存储ds_dict
with open("./corpus/ds_dict_balanced.pkl", "wb") as f:
    pickle.dump(ds_dict, f)


Balancing training split for dataset:  BBBP task p_np
原始: Total=1631, Pos=1341 , Neg=290
复制负样本: 290 -> 1341
Processing BBBP_p_np split: train


Map (num_proc=48): 100%|██████████| 2682/2682 [00:02<00:00, 1113.90 examples/s]


BBBP_p_np(TRAIN): Total=2682, Pos=1341 (50.00%), Neg=1341 (50.00%)
Balancing training split for dataset:  Tox21 task NR-AR
原始: Total=6258, Pos=250 , Neg=6008
复制正样本: 250 -> 1201
Processing Tox21_NR-AR split: train


Map (num_proc=48): 100%|██████████| 7209/7209 [00:03<00:00, 2256.43 examples/s]


Tox21_NR-AR(TRAIN): Total=7209, Pos=1201 (16.66%), Neg=6008 (83.34%)
Balancing training split for dataset:  Tox21 task NR-AR-LBD
原始: Total=6258, Pos=193 , Neg=6065
复制正样本: 193 -> 1213
Processing Tox21_NR-AR-LBD split: train


...

Map (num_proc=48): 100%|██████████| 148/148 [00:01<00:00, 83.23 examples/s] 


Processing HIV_HIV_active split: test


Map (num_proc=48): 100%|██████████| 4112/4112 [00:02<00:00, 1649.87 examples/s]


Processing BACE_Class split: test


Map (num_proc=48): 100%|██████████| 152/152 [00:01<00:00, 94.11 examples/s] 


In [3]:
map1 = {
    "p_np": (
        "Does this molecule have blood-brain barrier permeability (BBB penetration)? True for BBB permeable and False for not BBB permeable.",
        "True",
        "False"
    )
}
map2 = {
    "NR-AR": (
        "Does this molecule activate the androgen receptor (NR-AR)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-AR-LBD": (
        "Does this molecule activate the ligand-binding domain of the androgen receptor (NR-AR-LBD)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-AhR": (
        "Does this molecule activate the aryl hydrocarbon receptor (NR-AhR)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-Aromatase": (
        "Does this molecule inhibit the aromatase enzyme (NR-Aromatase)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-ER": (
        "Does this molecule activate the estrogen receptor (NR-ER)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-ER-LBD": (
        "Does this molecule activate the ligand-binding domain of the estrogen receptor (NR-ER-LBD)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-PPAR-gamma": (
        "Does this molecule activate the peroxisome proliferator-activated receptor gamma (NR-PPAR-gamma)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-ARE": (
        "Does this molecule activate the antioxidant response element (SR-ARE)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-ATAD5": (
        "Does this molecule activate ATAD5 signaling (SR-ATAD5)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-HSE": (
        "Does this molecule activate the heat shock element (SR-HSE)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-MMP": (
        "Does this molecule activate the mitochondrial membrane potential stress response (SR-MMP)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-p53": (
        "Does this molecule activate the p53 stress response pathway (SR-p53)? True for active and False for inactive.",
        "True",
        "False"
    )
}
map3 = {
    "FDA_APPROVED": (
        "Has this molecule been approved by the FDA? True for FDA approved and false for non-approved.",
        "True",
        "False"
    ),
    "CT_TOX": (
        "Is this molecule associated with clinical toxicity? True for clinically toxic and false for non-toxic.",
        "True",
        "False"
    )
}
map4 = {
    "HIV_active": (
        "Does this molecule inhibit HIV replication? True for active molecules which can inhibit HIV and false for inactive ones.",
        "True",
        "False"
    )
}
map5 = {
    "Class": (
        "Is the binding result of the molecular on beta-secretase 1 true or false?",
        "True",
        "False"
    )
}
# for dname,mapping in zip(['BBBP', 'Tox21', 'ClinTox', 'HIV', 'BACE'],[map1,map2,map3,map4,map5]):
#     print(dname,len(mapping))

# 将多个mapping合为一个
all_mapping = {**map1, **map2, **map3, **map4, **map5}
# print(len(all_mapping))

save_path = "./phase1/datasets/all/codes_map.pt"
codes_map = torch.load(save_path,weights_only=True)
label_desc_map = all_mapping

datasets_to_load = ['BBBP', 'Tox21', 'ClinTox', 'HIV', 'BACE']
prompt_template = """
You are a chemistry expert. Classify the given molecule into the correct category based on its molecular structure (token) and smiles expression. Each structure token represents a unique graph pattern (e.g., a kind of similar molecular graphs).
[Molecule] {text_attribute}
[Structure Token] <struct>{code}</struct>
[Task] {task_desc} Output the complete correct answer from the following two options:
1. {category_1}
2. {category_2}
[Answer]"""

tool,assistant_name = get_tokenizer(tokenizer_path=reload_path)
def process(example):
    # smiles example['text']
    text_attribute = example['smiles']
    task = example['task']
    code_number = example['code']
    code = f'<gstruct_{code_number}>'
    task_details = label_desc_map[task]
    example['instruction'] = prompt_template.format(
        text_attribute = text_attribute,
        code = code,
        task_desc = task_details[0],
        category_1 = task_details[1],        
        category_2 = task_details[2]
    )
    example['response'] = [task_details[1] if example['label'] == 1 else task_details[2]][0]
    messages = [
        {"role": "user", "content": example['instruction']},
        {"role": assistant_name, "content": example['response']},
    ]
    example["text"] = tool.apply_chat_template(messages, tokenize=False)+"<|end_of_text|>"
   
    return example

def seal_datasets(split='train'):
    datasets = {}
    start = 0
    for dataset_name in datasets_to_load:
        dataset_path = f"./phase1/datasets/{dataset_name}/graph_attributes.pkl"
        with open(dataset_path, "rb") as f:
            graphs = pickle.load(f)
        tar_graphs = graphs[split]

        tar_len = len(tar_graphs)
        tar_codes = list(codes_map[split].values())[start: start + tar_len]
        start += tar_len
        assert len(tar_codes) == tar_len,'Length mismatch between tar_graphs and tar_codes'
        for g,code in zip(tar_graphs,tar_codes):
            g['code'] = code
        
        tasks = g['label'].keys()
        for task in tasks:
            # print('Processing '+task+' for dataset: ',dataset_name)
            new_graphs = copy.deepcopy(tar_graphs)
            for g in new_graphs:
                g['label'] = g['label'][task]
                g['task'] = task

            balanced_graphs = new_graphs

            raw_dataset = Dataset.from_list(balanced_graphs)
            raw_dataset = raw_dataset.add_column("smiles", raw_dataset["text"])
            raw_dataset = raw_dataset.remove_columns("text")

            datasets[dataset_name+'_'+task] = raw_dataset.map(
                process,
                num_proc= multiprocessing.cpu_count(),
                load_from_cache_file=False,
                remove_columns=raw_dataset.column_names,
            )
            # === 统计正负样本数量和比例 ===
            # labels = raw_dataset["label"]
            # pos = sum(1 for l in labels if l == 1)
            # neg = sum(1 for l in labels if l == 0)
            # total = len(labels)
            # pos_ratio = pos / total if total > 0 else 0
            # neg_ratio = neg / total if total > 0 else 0
            # logger.info(f"{dataset_name}_{task}: Total={total}, Pos={pos} ({pos_ratio:.2%}), Neg={neg} ({neg_ratio:.2%})")
            # print(f"{dataset_name}_{task}: Total={total}, Pos={pos} ({pos_ratio:.2%}), Neg={neg} ({neg_ratio:.2%})")

    return datasets

splits = ['train','valid','test']


ds_dict = {}
for split in splits:
    ds_dict[split] = seal_datasets(split)

# 存储ds_dict
with open("./corpus/ds_dict_ori.pkl", "wb") as f:
    pickle.dump(ds_dict, f)


Map (num_proc=128): 100%|██████████| 152/152 [00:02<00:00, 60.79 examples/s]


In [ ]:
map1 = {
    "p_np": (
        "Does this molecule have blood-brain barrier permeability (BBB penetration)? True for BBB permeable and False for not BBB permeable.",
        "True",
        "False"
    )
}
map2 = {
    "NR-AR": (
        "Does this molecule activate the androgen receptor (NR-AR)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-AR-LBD": (
        "Does this molecule activate the ligand-binding domain of the androgen receptor (NR-AR-LBD)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-AhR": (
        "Does this molecule activate the aryl hydrocarbon receptor (NR-AhR)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-Aromatase": (
        "Does this molecule inhibit the aromatase enzyme (NR-Aromatase)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-ER": (
        "Does this molecule activate the estrogen receptor (NR-ER)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-ER-LBD": (
        "Does this molecule activate the ligand-binding domain of the estrogen receptor (NR-ER-LBD)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "NR-PPAR-gamma": (
        "Does this molecule activate the peroxisome proliferator-activated receptor gamma (NR-PPAR-gamma)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-ARE": (
        "Does this molecule activate the antioxidant response element (SR-ARE)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-ATAD5": (
        "Does this molecule activate ATAD5 signaling (SR-ATAD5)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-HSE": (
        "Does this molecule activate the heat shock element (SR-HSE)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-MMP": (
        "Does this molecule activate the mitochondrial membrane potential stress response (SR-MMP)? True for active and False for inactive.",
        "True",
        "False"
    ),
    "SR-p53": (
        "Does this molecule activate the p53 stress response pathway (SR-p53)? True for active and False for inactive.",
        "True",
        "False"
    )
}
map3 = {
    "FDA_APPROVED": (
        "Has this molecule been approved by the FDA? True for FDA approved and false for non-approved.",
        "True",
        "False"
    ),
    "CT_TOX": (
        "Is this molecule associated with clinical toxicity? True for clinically toxic and false for non-toxic.",
        "True",
        "False"
    )
}
map4 = {
    "HIV_active": (
        "Does this molecule inhibit HIV replication? True for active molecules which can inhibit HIV and false for inactive ones.",
        "True",
        "False"
    )
}
map5 = {
    "Class": (
        "Is the binding result of the molecular on beta-secretase 1 true or false?",
        "True",
        "False"
    )
}
# for dname,mapping in zip(['BBBP', 'Tox21', 'ClinTox', 'HIV', 'BACE'],[map1,map2,map3,map4,map5]):
#     print(dname,len(mapping))

# 将多个mapping合为一个
all_mapping = {**map1, **map2, **map3, **map4, **map5}
# print(len(all_mapping))

save_path = "./phase1/datasets/all/codes_map.pt"
codes_map = torch.load(save_path,weights_only=True)
label_desc_map = all_mapping

datasets_to_load = ['BBBP', 'Tox21', 'ClinTox', 'HIV', 'BACE']
prompt_template = """
You are a chemistry expert. Classify the given molecule into the correct category based on its molecular structure (token) and smiles expression. Each structure token represents a unique graph pattern (e.g., a kind of similar molecular graphs).
[Molecule] {text_attribute}
[Structure Token] <struct>{code}</struct>
[Task] {task_desc} Output the complete correct answer from the following two options:
1. {category_1}
2. {category_2}
[Answer]"""

tool,assistant_name = get_tokenizer(tokenizer_path=reload_path)
def process(example):
    # smiles example['text']
    text_attribute = example['smiles']
    task = example['task']
    code_number = example['code']
    code = f'<gstruct_{code_number}>'
    task_details = label_desc_map[task]
    example['instruction'] = prompt_template.format(
        text_attribute = text_attribute,
        code = code,
        task_desc = task_details[0],
        category_1 = task_details[1],        
        category_2 = task_details[2]
    )
    example['response'] = [task_details[1] if example['label'] == 1 else task_details[2]][0]
    messages = [
        {"role": "user", "content": example['instruction']},
        {"role": assistant_name, "content": example['response']},
    ]
    example["text"] = tool.apply_chat_template(messages, tokenize=False)+"<|end_of_text|>"
   
    return example

def seal_datasets(split='train'):
    datasets = {}
    start = 0
    for dataset_name in datasets_to_load:
        dataset_path = f"./phase1/datasets/{dataset_name}/graph_attributes.pkl"
        with open(dataset_path, "rb") as f:
            graphs = pickle.load(f)
        tar_graphs = graphs[split]

        tar_len = len(tar_graphs)
        tar_codes = list(codes_map[split].values())[start: start + tar_len]
        start += tar_len
        assert len(tar_codes) == tar_len,'Length mismatch between tar_graphs and tar_codes'
        for g,code in zip(tar_graphs,tar_codes):
            g['code'] = code
        
        tasks = g['label'].keys()
        for task in tasks:
            # print('Processing '+task+' for dataset: ',dataset_name)
            new_graphs = copy.deepcopy(tar_graphs)
            for g in new_graphs:
                g['label'] = g['label'][task]
                g['task'] = task

            if split == 'train':
                pos_samples = [g for g in new_graphs if g['label'] == 1]
                neg_samples = [g for g in new_graphs if g['label'] == 0]
                num_pos, num_neg = len(pos_samples), len(neg_samples)

                if num_pos < num_neg:
                    # 复制正样本
                    print(f"复制正样本: {num_pos} -> {num_neg}")
                    repeat_factor = num_neg // num_pos
                    remainder = num_neg % num_pos
                    pos_samples_extended = pos_samples * repeat_factor + random.sample(pos_samples, remainder)
                    balanced_graphs = pos_samples_extended + neg_samples
                    random.shuffle(balanced_graphs)
                elif num_neg < num_pos:
                    # 复制负样本
                    print(f"复制负样本: {num_neg} -> {num_pos}")
                    repeat_factor = num_pos // num_neg
                    remainder = num_pos % num_neg
                    neg_samples_extended = neg_samples * repeat_factor + random.sample(neg_samples, remainder)
                    balanced_graphs = pos_samples + neg_samples_extended
                    random.shuffle(balanced_graphs)
                else:
                    balanced_graphs = new_graphs
            else:
                balanced_graphs = new_graphs

            raw_dataset = Dataset.from_list(balanced_graphs)
            raw_dataset = raw_dataset.add_column("smiles", raw_dataset["text"])
            raw_dataset = raw_dataset.remove_columns("text")

            datasets[dataset_name+'_'+task] = raw_dataset.map(
                process,
                num_proc= multiprocessing.cpu_count(),
                load_from_cache_file=False,
                remove_columns=raw_dataset.column_names,
            )
            # === 统计正负样本数量和比例 ===
            # labels = raw_dataset["label"]
            # pos = sum(1 for l in labels if l == 1)
            # neg = sum(1 for l in labels if l == 0)
            # total = len(labels)
            # pos_ratio = pos / total if total > 0 else 0
            # neg_ratio = neg / total if total > 0 else 0
            # logger.info(f"{dataset_name}_{task}: Total={total}, Pos={pos} ({pos_ratio:.2%}), Neg={neg} ({neg_ratio:.2%})")
            # print(f"{dataset_name}_{task}: Total={total}, Pos={pos} ({pos_ratio:.2%}), Neg={neg} ({neg_ratio:.2%})")

    return datasets

splits = ['train','valid','test']
ds_dict = {}
for split in splits:
    ds_dict[split] = seal_datasets(split)

# 存储ds_dict
with open("./corpus/ds_dict.pkl", "wb") as f:
    pickle.dump(ds_dict, f)

Map (num_proc=128): 100%|██████████| 1631/1631 [00:01<00:00, 1145.76 examples/s]


BBBP_p_np: Total=1631, Pos=1341 (82.22%), Neg=290 (17.78%)


Map (num_proc=128): 100%|██████████| 12016/12016 [00:02<00:00, 4882.88 examples/s]


Tox21_NR-AR: Total=12016, Pos=6008 (50.00%), Neg=6008 (50.00%)


Map (num_proc=128): 100%|██████████| 12130/12130 [00:02<00:00, 4126.86 examples/s]


...

ClinTox_CT_TOX: Total=148, Pos=10 (6.76%), Neg=138 (93.24%)


Map (num_proc=128): 100%|██████████| 4112/4112 [00:01<00:00, 2272.41 examples/s]


HIV_HIV_active: Total=4112, Pos=130 (3.16%), Neg=3982 (96.84%)


Map (num_proc=128): 100%|██████████| 152/152 [00:01<00:00, 106.31 examples/s]


BACE_Class: Total=152, Pos=92 (60.53%), Neg=60 (39.47%)


In [ ]:
# ds_dict_ori：原始数据比例
# ds_dict_balanced: 1:5平衡后的数据比例
# ds_dict: 1:1平衡后的数据比例


In [ ]:
from collections import defaultdict
import torch
import pickle

save_path = "./phase1/datasets/all/codes_map.pt"
codes_map = torch.load(save_path,weights_only=True)
datasets_to_load = ['BBBP', 'HIV', 'BACE', 'ClinTox','Tox21']

# 生成 SMILES -> code 的映射字典
# Returns:
#     dict: {smiles_string: code}

smiles_to_code = defaultdict(dict)
for split in ['train', 'valid', 'test']:
    tar_len = 0
    start = 0
    for dataset_name in datasets_to_load:
        dataset_path = f"./phase1/datasets/{dataset_name}/graph_attributes.pkl"
        with open(dataset_path, "rb") as f:
            graphs = pickle.load(f)  # graphs 是一个 dict，里面按 split 存数据
        
        tar_graphs = graphs[split]
        tar_len = len(tar_graphs)
        print(f"for {split} of {dataset_name}: length -- {tar_len}; start -- {start}.")

        tar_codes = list(codes_map[split].values())[start: start + tar_len]
        start += tar_len

        assert len(tar_codes) == tar_len, \
            f"Length mismatch: graphs={tar_len}, codes={len(tar_codes)} for dataset {dataset_name}"

        for g, code in zip(tar_graphs, tar_codes):
            # 假设 g 里有 'smiles' 字段，如果是别的字段名需要改
            smiles_to_code[dataset_name][g['text']] = code



for train of BBBP: length -- 1631; start -- 0.
for train of Tox21: length -- 6258; start -- 1631.
for train of ClinTox: length -- 1184; start -- 7889.
for train of HIV: length -- 32896; start -- 9073.
for train of BACE: length -- 1210; start -- 41969.
for valid of BBBP: length -- 204; start -- 0.
for valid of Tox21: length -- 782; start -- 204.
for valid of ClinTox: length -- 148; start -- 986.
for valid of HIV: length -- 4112; start -- 1134.
for valid of BACE: length -- 151; start -- 5246.
for test of BBBP: length -- 204; start -- 0.
for test of Tox21: length -- 783; start -- 204.
for test of ClinTox: length -- 148; start -- 987.
for test of HIV: length -- 4112; start -- 1135.
for test of BACE: length -- 152; start -- 5247.


In [ ]:
save_path = './corpus/smiles_to_code.pkl'
with open(save_path, "wb") as f:
    pickle.dump(smiles_to_code, f)
print(f"SMILES->code 映射已保存到 {save_path}")

with open(save_path, "rb") as f:
    smiles_to_code = pickle.load(f)
print(f"SMILES->code 映射已加载自 {save_path}")
len(smiles_to_code['BACE'])


SMILES->code 映射已保存到 ./corpus/smiles_to_code.pkl
SMILES->code 映射已加载自 ./corpus/smiles_to_code.pkl


1513

🌟 构造ablation study的3种语料：去掉 strcut_code；struct_code 固定替换；struct_code 随机替换

In [ ]:
import re
import pickle

data_path = "corpus/ds_dict_ori.pkl"
with open(data_path, "rb") as f:
    ds_dict = pickle.load(f)
test_datasets = ds_dict['test']

pattern = r'(\[Structure Token\] <struct><gstruct_)\d+(></struct>\n?)'

def replace_with_fixed_number(match):
    return f"{match.group(1)}999{match.group(2)}"

def clean(example):
    example['instruction'] = re.sub(pattern, replace_with_fixed_number, example['instruction'])
    example['text'] = re.sub(pattern, replace_with_fixed_number, example['text'])
    return example

for k, td in test_datasets.items():
    test_datasets[k] = td.map(clean)
for k, td in test_datasets.items():
    continue
with open("./corpus/test_datasets_999.pkl", "wb") as f:
    pickle.dump(test_datasets, f)

In [ ]:
import re
import pickle
import random
random.seed(42)

data_path = "corpus/ds_dict_ori.pkl"
with open(data_path, "rb") as f:
    ds_dict = pickle.load(f)
test_datasets = ds_dict['test']

pattern = r'(\[Structure Token\] <struct><gstruct_)\d+(></struct>\n?)'

def replace_with_random(match):
    rand_num = random.randint(1, 999999)
    return f"{match.group(1)}{rand_num}{match.group(2)}"

def clean(example):
    example['instruction'] = re.sub(pattern, replace_with_random, example['instruction'])
    example['text'] = re.sub(pattern, replace_with_random, example['text'])
    return example

for k, td in test_datasets.items():
    test_datasets[k] = td.map(clean)
for k, td in test_datasets.items():
    continue
with open("./corpus/test_datasets_random.pkl", "wb") as f:
    pickle.dump(test_datasets, f)

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

Map: 100%|██████████| 152/152 [00:00<00:00, 4856.11 examples/s]


{'instruction': '\nYou are a chemistry expert. Classify the given molecule into the correct category based on its molecular structure (token) and smiles expression. Each structure token represents a unique graph pattern (e.g., a kind of similar molecular graphs).\n[Molecule] O1CCC(CC1)CNC(=O)C(Cc1cc2cc(ccc2nc1N)-c1ccccc1C)C\n[Structure Token] <struct><gstruct_63661></struct>\n[Task] Is the binding result of the molecular on beta-secretase 1 true or false? Output the complete correct answer from the following two options:\n1. True\n2. False\n[Answer]',
 'response': 'True',
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful and reliable assistant.<|eot_id|><|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nYou are a chemistry expert. Classify the given molecule into the correct category based on its molecular structure (token) and smiles expression. Each structure token represents a unique graph pattern (e.g., a kind of similar molecular 

In [ ]:
import pickle
import re

pattern = r'\[Structure Token\] <struct><gstruct_\d+></struct>\n?'
def clean(example):
    example['instruction'] = re.sub(pattern, '', example['instruction'])
    example['text'] = re.sub(pattern, '', example['text'])
    return example

data_path = "corpus/ds_dict_ori.pkl"
with open(data_path, "rb") as f:
    ds_dict = pickle.load(f)
ds_dict_puretext = dict()
for split, datasets in ds_dict.items():
    for k, td in datasets.items():
        datasets[k] = td.map(clean)
    ds_dict_puretext[split] = datasets
with open("./corpus/ds_dict_puretext.pkl", "wb") as f:
    pickle.dump(ds_dict_puretext, f)

Map: 100%|██████████| 152/152 [00:00<00:00, 5158.84 examples/s]
